L2-Regularization Method on The Blood-brain barrier penetration (BBBP)
--------------------------------------------------------------

### All libraries we need

In [3]:
import warnings
warnings.filterwarnings('ignore')


import os
import os.path as osp
import shutil
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import copy
import time
import statistics as stat
import psutil
import itertools
import tracemalloc
import gc


import torch
import torch.nn as nn
from torch.optim import Adam
from  utils import *

#********************************************************#
'''
load_dataset contain lots of functions for loading several datasets and 
also there is a function as name get_ dataloader for generating a
dictionary of training, validation, and testing dataLoader.
'''
from load_dataset import get_dataset, get_dataloader

#********************************************************#
'''
As we need several arguments for training process, we store all argument in configure file. 
For using this file, you need the library'Typed Argument Parser (Tap). So you need 'pip install typed-argument-parser'. 
'''
from Configures import data_args, train_args, model_args

### Regularization Rate
#### Regularization rates range from the following numbers:

In [4]:

0, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2,0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1, 1e2, 1e3, 1e6
;

''

### Functions for Measuring criterias

In [5]:
def get_num_parameters(model: nn.Module, count_nonzero_only=False) -> int:
    """
    calculate the total number of parameters of model
    :param count_nonzero_only: only count nonzero weights
    """
    num_counted_elements = 0
    for param in model.parameters():
        if count_nonzero_only:
            num_counted_elements += param.count_nonzero()
        else:
            num_counted_elements += param.numel()
    return num_counted_elements

# Function to get CPU usage
def get_cpu_usage():
    return psutil.cpu_percent(interval=1)



# Function to approximate power consumption (Assume some average power usage per CPU percentage point)
def estimate_power_usage(cpu_usage):
    base_power_usage = 10  # Assumed base power usage in watts
    power_per_percent = 0.5  # Assumed additional watts per CPU usage percent
    return base_power_usage + (power_per_percent * cpu_usage)

# The model size based on the number of parameters
def get_model_size_param(model: nn.Module, data_width=32, count_nonzero_only=False) -> int:
    """
    calculate the model size in bits
    :param data_width: #bits per element
    :param count_nonzero_only: only count nonzero weights
    """
    return get_num_parameters(model, count_nonzero_only) * data_width



### start loading data

In [6]:
print(data_args.dataset_name)
print(data_args.dataset_dir)

bbbp
/datasets


In [7]:
dataset = get_dataset(data_args.dataset_dir, data_args.dataset_name)
input_dim = dataset.num_node_features
output_dim = int(dataset.num_classes)


print(input_dim)
print(output_dim)

9
2


### Data Analysis

In [8]:
avg_nodes = 0.0
avg_edge_index = 0.0
for i in range(len(dataset)):
    avg_nodes += dataset[i].x.shape[0]
    avg_edge_index += dataset[i].edge_index.shape[1]
avg_nodes /= len(dataset)
avg_edge_index /= len(dataset)
print(f"graphs {len(dataset)}, avg_nodes{avg_nodes :.4f}, avg_edge_index_{avg_edge_index/2 :.4f}")

best_acc = 0.0
data_size = len(dataset)
print(f'The total num of dataset is {data_size}')



graphs 2050, avg_nodes23.9356, avg_edge_index_25.8151
The total num of dataset is 2050


### Preprocessing and cleaning dataset

In [9]:
#cleaned_dataset = [graph for graph in dataset if graph.edge_index.numpy()!=[]]
cleaned_dataset = [graph for graph in dataset if graph.edge_index.numpy().size> 0]
cleaned_dataset_len=len(cleaned_dataset)
print(f'The number of graphs after cleaning dataset is: {cleaned_dataset_len}')

The number of graphs after cleaning dataset is: 2039


In [10]:
dataloader=get_dataloader(cleaned_dataset, batch_size=train_args.batch_size, random_split_flag=True, data_split_ratio=[0.8, 0.1, 0.1], seed=2)

### Model for Training

In [11]:
from GCN import GCNNet

def get_model(input_dim, output_dim, model_args):
    if model_args.model_name.lower() == 'gcn':
        return GCNNet(input_dim, output_dim, model_args)
    elif model_args.model_name.lower() == 'gat':
        return GATNet(input_dim, output_dim, model_args)
    elif model_args.model_name.lower() == 'gin':
        return GINNet(input_dim, output_dim, model_args)
    else:
        raise NotImplementedError
        


class GnnBase(nn.Module):
    def __init__(self):
        super(GnnBase, self).__init__()

    def forward(self, data):
        data = data.to(self.device)
        logits, prob, emb = self.model(data)
        return logits, prob, emb

    def update_state_dict(self, state_dict):
        original_state_dict = self.state_dict()
        loaded_state_dict = dict()
        for k, v in state_dict.items():
            if k in original_state_dict.keys():
                loaded_state_dict[k] = v
        self.load_state_dict(loaded_state_dict)

    def to_device(self):
        self.to(self.device)

    def save_state_dict(self):
        pass


class GnnNets(GnnBase):
    def __init__(self, input_dim, output_dim, model_args):
        super(GnnNets, self).__init__()
        self.model = get_model(input_dim, output_dim, model_args)
        self.device = model_args.device

    def forward(self, data):
        data = data.to(self.device)
        logits, prob, emb = self.model(data)
        return logits, prob, emb



### Functions requirement training process

In [32]:
def evaluate_GC(eval_dataloader,model, criterion):
    acc = []
    loss_list = []
    model.eval()
    with torch.no_grad():
        for batch in eval_dataloader:
            logits, probs, _ = model(batch)
            loss = criterion(logits, batch.y)

            ## record
            _, prediction = torch.max(logits, -1)
            loss_list.append(loss.item())
            acc.append(prediction.eq(batch.y).cpu().numpy())

        eval_state = {'loss': np.average(loss_list),
                      'acc': np.concatenate(acc, axis=0).mean()}

    return eval_state


def test_GC(test_dataloader,model, criterion):
    acc = []
    loss_list = []
    pred_probs = []
    predictions = []
    model.eval()
    with torch.no_grad():
        for batch in test_dataloader:
            logits, probs, _ = model(batch)
            loss = criterion(logits, batch.y)

            # record
            _, prediction = torch.max(logits, -1)
            loss_list.append(loss.item())
            acc.append(prediction.eq(batch.y).cpu().numpy())
            predictions.append(prediction)
            pred_probs.append(probs)

    test_state = {'loss': np.average(loss_list),
                  'acc': np.average(np.concatenate(acc, axis=0).mean())}

    pred_probs = torch.cat(pred_probs, dim=0).cpu().detach().numpy()
    predictions = torch.cat(predictions, dim=0).cpu().detach().numpy()
    return test_state, pred_probs, predictions

def train(l2):
    logits, probs, _ = model(batch)
    loss = criterion(logits, batch.y)
    l2_reg = torch.tensor(0.)
    for param in model.parameters():
        l2_reg += torch.norm(param)

    # Combine the loss function with L2 regularization
    loss += (l2 * l2_reg)
    
    # optimization
    optimizer.zero_grad()
    loss.backward()
    torch.nn.utils.clip_grad_value_(model.parameters(), clip_value=2.0)
    optimizer.step()
    
    ## record
    _, prediction = torch.max(logits, -1)
    loss_list.append(loss.item())
    acc.append(prediction.eq(batch.y).cpu().numpy())
    
  
    return acc



### save path for model

In [29]:

if not os.path.isdir('checkpoint'):
    os.mkdir('checkpoint')
if not os.path.isdir(os.path.join('checkpoint', data_args.dataset_name)):
    os.mkdir(os.path.join('checkpoint', f"{data_args.dataset_name}"))
ckpt_dir = f"./checkpoint/{data_args.dataset_name}/"



def save_best(ckpt_dir, epoch, state, model_name, eval_acc, is_best, is_reg):
    print('saving....')
    model.to_device()
    state_save = {
        'net':state,
        'epoch':epoch,
        'acc': eval_acc 
        }
    best_pth_name = f'{model_name}_best.pth'
    reg_pth_name = f'{model_name}_reg_best.pth'
    
  
    if is_reg & is_best:
        ckpt_path = os.path.join(ckpt_dir, reg_pth_name) 
        torch.save(state_save, ckpt_path)
    
     
    if is_reg == False & is_best:
        ckpt_path = os.path.join(ckpt_dir, best_pth_name)  
        torch.save(state_save, ckpt_path)
           
        
    model.to_device()

### Measurement

#### Setting Regularization rate

In [30]:

# This is a dictionary to save all measurements. Aftre measuring, we can compute mean and std of each item.
Eva_final=dict()

# The following are all list of criteria for measurements. 
# We collect all desired datas of each list across iterations. 
# Then, we compute average and std of each list.

#Base model
Base_model_accuracy=[]
T_base_model=[]
Num_parm_base_model=[]
Base_model_size=[]
Base_Energy_Consumption=[]
Base_Cpu_Usage=[]
Base_Memory_Usage=[]

#regularized model
Reg_model_accuracy=[]
T_Reg_model=[]
Num_parm_Reg_model=[]
Reg_model_size=[]
Reg_Energy_Consumption=[]
Reg_Cpu_Usage=[]
Reg_Memory_Usage=[]

# Here is the dictionary to record the list of all measurements
Eva_measure={'base model accuracy':Base_model_accuracy,
            'time inference of base model':T_base_model,
            'number parmameters of base model':Num_parm_base_model,
            'base model size':Base_model_size,
            'energy consumption of base model':Base_Energy_Consumption,
            'cpu usage of base model':Base_Cpu_Usage,
            'memory usage of base model':Base_Memory_Usage,
            'regularized model accuracy': Reg_model_accuracy,
            'time inference of regularized model':T_Reg_model,
            'number parmameters of regularized model':Num_parm_Reg_model,
            'regularized model size':Reg_model_size,
            'energy consumption of regularized model':Reg_Energy_Consumption,
            'cpu usage of regularized model':Reg_Cpu_Usage,
            'memory usage of regularized model':Reg_Memory_Usage
            }

             


In [ ]:
### Regularization Rate
l2_lambda = 0.9

# The number of epoch
num_epochs=100
# The number of iteration
num_iterations=10
# Parameter for early stopping
train_args.early_stopping=num_epochs*0.2

### Training, and Regularization

In [38]:

for i in range(num_iterations):
        print('________________________________________________')
        print('************************************************')
        
        print(f'Training and evaluation before regularization ')
        print("Starting training...")

        model = GnnNets(input_dim, output_dim, model_args)
        model.to_device()
        criterion = nn.CrossEntropyLoss()
        optimizer = Adam(model.parameters(), lr=train_args.learning_rate, weight_decay=train_args.weight_decay)

       
        Eva=dict()# It is a dictionary to arrange output of this iteration
        best_acc=0  
        is_reg=False
        reg_rate=0
            
       
        #Training base model
        print(f"The regularization rate is :{reg_rate}")
        for epoch in range(num_epochs):  
            acc=[]
            loss_list = []
            model.train()
            for batch in dataloader['train']:
                acc=train(reg_rate)        

            eval_state = evaluate_GC(dataloader['eval'], model, criterion)
            acc_eval=eval_state['acc']
            # report train msg
            if epoch % 20 == 0:   
                print(f"Eval Epoch: {epoch} | Loss: {eval_state['loss']:.3f} | Acc: {eval_state['acc']:.3f}")


            # only save the best model
            if eval_state['acc'] > best_acc:
                early_stop_count = 0
            else:
                early_stop_count += 1

            if early_stop_count > train_args.early_stopping:
                break
            is_best = (eval_state['acc'] > best_acc)
            if is_best:
                best_acc = eval_state['acc']
                early_stop_count = 0
                    
            if is_best or epoch % train_args.save_epoch == 0:
                 save_best(ckpt_dir, epoch, model.state_dict(), model_args.model_name, eval_state['acc'], is_best, is_reg)   
                                      



        #### load the best model
        base_model_path = os.path.join(ckpt_dir, f'{model_args.model_name}_best.pth') 
        checkpoint = torch.load(base_model_path)
        model.load_state_dict(checkpoint['net'])
        recover_model = lambda: model.load_state_dict(checkpoint['net'])

        # Start monitoring CPU and memory usage, model size, number of parametes, time inference and  power consumption
        gc.collect()
        time.sleep(5)  # Add a 5-second delay to stabilize the initial state
        tracemalloc.start()  # Start tracking memory allocations
        snapshot_before = tracemalloc.take_snapshot()#take a snapshot of the current memory state before starting the measurement.

        t0 = time.perf_counter()
        initial_cpu_usage = get_cpu_usage()
        power_usage = estimate_power_usage(initial_cpu_usage)

        test_state, _, _ = test_GC(dataloader['test'], model, criterion)
        base_model_accuracy= test_state['acc']

        base_cpu_usage = get_cpu_usage()
        t1 = time.perf_counter()
        t_base_model=t1-t0

        snapshot_after = tracemalloc.take_snapshot()
        tracemalloc.stop()
        top_stats = snapshot_after.compare_to(snapshot_before, 'lineno')

        base_total_memory_diff = sum([stat.size_diff for stat in top_stats])
        
        base_energy_consumption = power_usage * t_base_model
        # model size
        base_model_size = os.path.getsize(base_model_path)
        # number of parameters
        num_parm_base_model=get_num_parameters(model, count_nonzero_only=True)

        gc.collect()
        time.sleep(5) 
        
        print(f'*****Results of base model*********')

        print(f"base model has accuracy on test set={base_model_accuracy:.2f}%")
        print(f"base model has size={base_model_size:.2f} bytes")
        print(f"The time inference of base model is ={t_base_model}") 
        print(f"The number of parametrs of base model is:{num_parm_base_model}") 

        print(f"Energy Consumption : {base_energy_consumption:.3f}")
        print(f"total memory usage of base model':{base_total_memory_diff} ")
        print(f"cpu usage of base model':{base_cpu_usage:.3f} %")


        #Update Eva dictionary
        Eva.update({'base model accuracy': base_model_accuracy,
                    'time inference of base model': t_base_model,
                    'number parmameters of base model': num_parm_base_model,
                    'size of base model': base_model_size, 
                    'energy consumption of base model':base_energy_consumption,
                    'total memory usage of base model':base_total_memory_diff,
                    'cpu usage of base model':base_cpu_usage
                   })

        gc.collect()
        time.sleep(5)  

        #### Regularization of the Model
        gc.collect()
        time.sleep(5)   

        print('________*******************************_____________')
        print(f'Regularized Model')

        best_reg_checkpoint = dict()
        best_reg_acc = 0
        is_reg=True
        reg_rate=l2_lambda
            
        
        model = GnnNets(input_dim, output_dim, model_args)
        model.to_device()
        criterion = nn.CrossEntropyLoss()
        optimizer = Adam(model.parameters(), lr=train_args.learning_rate, weight_decay=train_args.weight_decay)

        print(f"The regularization rate is :{reg_rate}")
        for epoch in range(num_epochs):
            # At the end of each train iteration, we have to apply the pruning mask
            #    to keep the model sparse during the training
            acc=[]
            loss_list = []

            for batch in dataloader['train']:
                acc=train(reg_rate)       

            eval_state = evaluate_GC(dataloader['eval'], model, criterion)
            acc_eval=eval_state['acc']

            # report train msg
            if epoch % 5 == 0:   
                print(f"Eval Epoch: {epoch} | Loss: {eval_state['loss']:.3f} | Acc: {eval_state['acc']:.3f}")


            # only save the best model
            if eval_state['acc'] > best_reg_acc :
                early_stop_count = 0
            else:
                early_stop_count += 1

            if early_stop_count > train_args.early_stopping:
                break
            is_best = (eval_state['acc'] > best_reg_acc )
           
            if is_best:
                best_acc = eval_state['acc']
                early_stop_count = 0
               
          
           
            if is_best or epoch % train_args.save_epoch == 0:
                
                save_best(ckpt_dir, epoch, model.state_dict(), model_args.model_name, eval_state['acc'], is_best, is_reg)


        #### load the best regularized model
        reg_pth_name = f'{model_args.model_name}_reg_best.pth'
        reg_model_path = os.path.join(ckpt_dir, reg_pth_name) 
        checkpoint = torch.load(reg_model_path)
        model.load_state_dict(checkpoint['net'])
        recover_model = lambda:model.load_state_dict(checkpoint['net'])


        # Result of regularization
      

        gc.collect()
        time.sleep(5)  
        tracemalloc.start() 
        snapshot_before = tracemalloc.take_snapshot()

        t0 = time.perf_counter()
        initial_cpu_usage = get_cpu_usage()
        power_usage = estimate_power_usage(initial_cpu_usage)

        test_state, _, _ = test_GC(dataloader['test'], model, criterion)
        regularized_model_accuracy= test_state['acc']

        regularized_cpu_usage = get_cpu_usage()
        t1 = time.perf_counter()
        t_regularized_model=t1-t0

        snapshot_after = tracemalloc.take_snapshot()
        tracemalloc.stop()
        top_stats = snapshot_after.compare_to(snapshot_before, 'lineno')

        regularized_total_memory_diff = sum([stat.size_diff for stat in top_stats])
        regularized_energy_consumption = power_usage * t_regularized_model
        regularized_model_size = os.path.getsize( reg_model_path )
        num_parm_regularized_model=get_num_parameters(model, count_nonzero_only=True)

        gc.collect()
        time.sleep(5)  # Add a 5-second delay to stabilize the initial state    
        
 
        
        print('****************Result of regularized model ******************')
    
        
        print(f"{l2_lambda} regularized model has accuracy on test set={regularized_model_accuracy:.2f}%")
        print(f"{l2_lambda} regularized model has size={regularized_model_size:.2f} bit")
        print(f"The time inference of {l2_lambda} regularized model is ={t_regularized_model}") 
        print(f"The number of parametrs of {l2_lambda} regularized model is:{num_parm_regularized_model}") 

        print(f"Energy Consumption of {l2_lambda} regularized model: {regularized_energy_consumption:.3f}")
        print(f"total memory usage of {l2_lambda} regularized model':{regularized_total_memory_diff} ")
        print(f"cpu usage of {l2_lambda} regularized model':{regularized_cpu_usage:.3f} %")


        #Update Eva dictionary
        Eva.update({'regularized model accuracy': regularized_model_accuracy,
                    'time inference of regularized model': t_regularized_model,
                    'number parmameters of regularized model': num_parm_regularized_model,
                    'size of regularized model': regularized_model_size, 
                    'energy consumption of regularized model':regularized_energy_consumption,
                    'total memory usage of regularized model':regularized_total_memory_diff,
                    'cpu usage of regularized model':regularized_cpu_usage
                   })

        gc.collect()
        time.sleep(5)   

        

        Base_model_accuracy.append(Eva['base model accuracy'])
        T_base_model.append(Eva['time inference of base model'])
        Num_parm_base_model.append(int(Eva['number parmameters of base model']))
        Base_model_size.append(int(Eva['size of base model']))
        Base_Energy_Consumption.append(Eva['energy consumption of base model'])
        Base_Cpu_Usage.append(Eva['cpu usage of base model'])
        Base_Memory_Usage.append(Eva['total memory usage of base model'])

        Reg_model_accuracy.append(Eva['regularized model accuracy'])
        T_Reg_model.append(Eva['time inference of regularized model'])
        Num_parm_Reg_model.append(int(Eva['number parmameters of regularized model']))
        Reg_model_size.append(int(Eva['size of regularized model']))
        Reg_Energy_Consumption.append(Eva['energy consumption of regularized model'])
        Reg_Cpu_Usage.append(Eva['cpu usage of regularized model'])
        Reg_Memory_Usage.append(Eva['total memory usage of regularized model'])



Training and evaluation before regularization 
Starting training...
The regularization rate is :0
Eval Epoch: 0 | Loss: 0.542 | Acc: 0.754
saving....
saving....
Eval Epoch: 5 | Loss: 0.473 | Acc: 0.808
saving....
saving....
Eval Epoch: 10 | Loss: 0.457 | Acc: 0.788
saving....
Eval Epoch: 15 | Loss: 0.417 | Acc: 0.847
saving....
*****Results of base model*********
base model has accuracy on test set=0.80%
base model has size=141721.00 bytes
The time inference of base model is =2.1644885999849066
The number of parametrs of base model is:34524
Energy Consumption : 120.346
total memory usage of base model':25760 
cpu usage of base model':89.400 %
________*******************************_____________
Regularized Model
The regularization rate is :100000
Eval Epoch: 0 | Loss: 0.693 | Acc: 0.754
saving....
saving....
saving....
saving....
saving....
Eval Epoch: 5 | Loss: 0.693 | Acc: 0.754
saving....
saving....
saving....
saving....
saving....
Eval Epoch: 10 | Loss: 0.693 | Acc: 0.246
saving...

### Computing Mean and Std of Lists

In [39]:
Eva_final=dict()
base_model_accuracy_mean = stat.mean(Base_model_accuracy)
base_model_accuracy_std =  stat.stdev(Base_model_accuracy)
Eva_final.update({'Ave of base model accuracy':float(format(base_model_accuracy_mean, '.3f'))})
Eva_final.update({'Std of base model accuracy':float(format(base_model_accuracy_std, '.3f'))})
base_model_accuracy = "{:.3f} ± {:.3f}".format(base_model_accuracy_mean ,base_model_accuracy_std)
print(f"Base model accuracy is:{base_model_accuracy}")

                 
t_base_model_mean =stat.mean(T_base_model)
t_base_model_std =stat.stdev(T_base_model)  
Eva_final.update({'Ave of time inference of base model':float(format(t_base_model_mean, '.3f'))})
Eva_final.update({'Std of time inference of base model':float(format(t_base_model_std, '.3f'))})
t_base_model = "{:.3f} ± {:.3f}".format(t_base_model_mean ,t_base_model_std)
print(f"Time inference of Base model :{t_base_model}")


num_parm_base_model_mean = stat.mean(Num_parm_base_model)
num_parm_base_model_std = stat.stdev(Num_parm_base_model)
Eva_final.update({'Ave of number parmameters of base model':num_parm_base_model_mean})
Eva_final.update({'Std of number parmameters of base model':num_parm_base_model_std})
num_parm_base_model = "{:.3f} ± {:.3f}".format(num_parm_base_model_mean ,num_parm_base_model_std)
print(f"Time number of parameters of Base model :{num_parm_base_model}")

base_model_size_mean = stat.mean(Base_model_size)
base_model_size_std = stat.stdev(Base_model_size)
Eva_final.update({'Ave of base model size':base_model_size_mean})
Eva_final.update({'Std of base model size':base_model_size_std})
base_model_size_model = "{:.3f} ± {:.3f}".format(base_model_size_mean ,base_model_size_std)
print(f"The size of Base model :{base_model_size} bytes")


base_energy_consumption_mean = stat.mean(Base_Energy_Consumption)
base_energy_consumption_std = stat.stdev(Base_Energy_Consumption)
Eva_final.update({'Ave of energy consumption of base model':base_energy_consumption_mean })
Eva_final.update({'Std of energy consumption of base model':base_energy_consumption_std})
base_energy_consumption = "{:.3f} ± {:.3f}".format(base_energy_consumption_mean ,base_energy_consumption_std)
print(f"The energy consumption of Base model :{base_energy_consumption} ")


base_cpu_usage_mean = stat.mean(Base_Cpu_Usage)
base_cpu_usage_std = stat.stdev(Base_Cpu_Usage)
Eva_final.update({'Ave of cpu usage of base model':base_cpu_usage_mean})
Eva_final.update({'Std of cpu usage of base model':base_cpu_usage_std})
base_cpu_usage = "{:.3f} ± {:.3f}".format(base_cpu_usage_mean ,base_cpu_usage_std)
print(f"The CPU usage of Base model :{base_cpu_usage} ")


base_memory_usage_mean = stat.mean(Base_Memory_Usage)
base_memory_usage_std = stat.stdev(Base_Memory_Usage)
Eva_final.update({'Ave of memory usage of base model':base_memory_usage_mean})
Eva_final.update({'Std of memory usage of base model':base_memory_usage_std})
base_memory_usage = "{:.3f} ± {:.3f}".format(base_memory_usage_mean ,base_memory_usage_std)
print(f"The memory usage of Base model :{base_memory_usage} ")

print(100 * "=")
####################################################

reg_model_accuracy_mean =stat.mean(Reg_model_accuracy)
reg_model_accuracy_std = stat.stdev(Reg_model_accuracy)
Eva_final.update({'Ave of regularized model accuracy':float(format(reg_model_accuracy_mean, '.3f'))})
Eva_final.update({'Std of regularized model accuracy':float(format(reg_model_accuracy_std, '.3f'))})
reg_model_accuracy = "{:.3f} ± {:.3f}".format(reg_model_accuracy_mean ,reg_model_accuracy_std)
print(f"Regularized model accuracy is:{reg_model_accuracy}")
                 

t_reg_model_mean = stat.mean(T_Reg_model)
t_reg_model_std =stat.stdev(T_Reg_model)
Eva_final.update({'Ave of time inference of regularized model':float(format(t_reg_model_mean, '.3f'))})
Eva_final.update({'Std of time inference of regularized model':float(format(t_reg_model_std, '.3f'))})
t_reg_model = "{:.3f} ± {:.3f}".format(t_reg_model_mean ,t_reg_model_std)
print(f"Time inference of Regularized model :{t_reg_model}")

num_parm_reg_model_mean = stat.mean(Num_parm_Reg_model)
num_parm_reg_model_std = stat.stdev(Num_parm_Reg_model)
Eva_final.update({'Ave of number parmameters of regularized model':num_parm_reg_model_mean})
Eva_final.update({'Std of number parmameters of regularized model':num_parm_reg_model_std})
num_parm_reg_model = "{:.3f} ± {:.3f}".format(num_parm_reg_model_mean ,num_parm_reg_model_std)
print(f"Time number of parameters of Regularized model :{num_parm_reg_model}")

reg_model_size_mean =stat.mean( Reg_model_size)
reg_model_size_std = stat.stdev(Reg_model_size)
Eva_final.update({'Ave of regularized model size':reg_model_size_mean})
Eva_final.update({'Std of regularized model size':reg_model_size_std })
reg_model_size = "{:.3f} ± {:.3f}".format(reg_model_size_mean ,reg_model_size_std)
print(f"The size of Regularized model :{reg_model_size} bytes")

reg_energy_consumption_mean = stat.mean(Reg_Energy_Consumption)
reg_energy_consumption_std = stat.stdev(Reg_Energy_Consumption)
Eva_final.update({'Ave of energy consumption of regularized model':reg_energy_consumption_mean })
Eva_final.update({'Std of energy consumption of regularized model':reg_energy_consumption_std})
reg_energy_consumption = "{:.3f} ± {:.3f}".format(reg_energy_consumption_mean ,reg_energy_consumption_std)
print(f"The energy consumption of Regularized model :{reg_energy_consumption} ")


reg_cpu_usage_mean = stat.mean(Reg_Cpu_Usage)
reg_cpu_usage_std = stat.stdev(Reg_Cpu_Usage)
Eva_final.update({'Ave of cpu usage of regularized model':reg_cpu_usage_mean})
Eva_final.update({'Std of cpu usage of regularized model':reg_cpu_usage_std})
reg_cpu_usage = "{:.3f} ± {:.3f}".format(reg_cpu_usage_mean ,reg_cpu_usage_std)
print(f"The CPU usage of Regularized model :{reg_cpu_usage} ")


reg_memory_usage_mean = stat.mean(Reg_Memory_Usage)
reg_memory_usage_std = stat.stdev(Reg_Memory_Usage)
#desc = "{:.3f} ± {:.3f}".format(acc_mean,acc_std)
Eva_final.update({'Ave of memory usage of regularized model':reg_memory_usage_mean})
Eva_final.update({'Std of memory usage of regularized model':reg_memory_usage_std})
reg_memory_usage = "{:.3f} ± {:.3f}".format(reg_memory_usage_mean ,reg_memory_usage_std)
print(f"The memory usage of Regularized model :{reg_memory_usage} ")



#################################


print(f"All measurement about regularization process of rate:{l2_lambda} ")   
Eva_final

Base model accuracy is:0.789 ± 0.010
Time inference of Base model :2.155 ± 0.047
Time number of parameters of Base model :34522.833 ± 4.951
The size of Base model :141721 bytes
The energy consumption of Base model :54.147 ± 34.414 
The CPU usage of Base model :27.258 ± 28.510 
The memory usage of Base model :26127.583 ± 489.902 
Regularized model accuracy is:0.656 ± 0.226
Time inference of Regularized model :2.139 ± 0.019
Time number of parameters of Regularized model :34535.833 ± 19.922
The size of Regularized model :141761.000 ± 0.000 bytes
The energy consumption of Regularized model :44.075 ± 25.982 
The CPU usage of Regularized model :16.442 ± 21.264 
The memory usage of Regularized model :23575.833 ± 870.471 
All measurement about regularization process of rate:100000 


{'Ave of base model accuracy': 0.789,
 'Std of base model accuracy': 0.01,
 'Ave of time inference of base model': 2.155,
 'Std of time inference of base model': 0.047,
 'Ave of number parmameters of base model': 34522.833333333336,
 'Std of number parmameters of base model': 4.951277765905637,
 'Ave of base model size': 141721,
 'Std of base model size': 0.0,
 'Ave of energy consumption of base model': 54.14708325008978,
 'Std of energy consumption of base model': 34.413852194055785,
 'Ave of cpu usage of base model': 27.258333333333333,
 'Std of cpu usage of base model': 28.510172294155367,
 'Ave of memory usage of base model': 26127.583333333332,
 'Std of memory usage of base model': 489.90248720505286,
 'Ave of regularized model accuracy': 0.656,
 'Std of regularized model accuracy': 0.226,
 'Ave of time inference of regularized model': 2.139,
 'Std of time inference of regularized model': 0.019,
 'Ave of number parmameters of regularized model': 34535.833333333336,
 'Std of number

### Recording results on txt file

In [40]:

dataset_name = 'BBBP'
Pruning_Method='Regularization'
max_epoch = 100
resume = True
result_folder ='pathresult/'
if not os.path.exists(result_folder):
    os.makedirs(result_folder)



file_name = result_folder+Pruning_Method+'_'+'with rate of regularization of'+'_'+str(l2_lambda)+'_on_'+dataset_name+'_'+str(max_epoch)+'.txt'

with open(file_name, 'w') as f:
        f.write('%s:%s\n'%('dataset_name', 'BBBP'))
        f.write('%s:%s\n'%('max_epoch', max_epoch))
        f.write('%s:%s\n'%('sparsity', l2_lambda))
        for key, value in Eva_final.items():
            f.write('%s:%s\n'%(key, value))
     
        for key, value in Eva_measure.items():
            f.write('%s:%s\n' % (key, ','.join(map(str, value)))) 
       
       